In [7]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import save_model
import random
import os

classifier = Sequential()

#Convolution step
classifier.add(Convolution2D(32, 3, 3, input_shape= (64, 64, 3), activation='relu'))
classifier.add(Convolution2D(32, 3, 3, activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(rate=random.uniform(0, .2)))


#Convolution step
classifier.add(Convolution2D(64, 3, 3, activation='relu'))
classifier.add(Convolution2D(64, 3, 3, activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(rate=random.uniform(0, .2)))

#Convolution step
classifier.add(Convolution2D(128, 3, 3, activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(rate=random.uniform(0, .2)))

#Flattening step
classifier.add(Flatten())
#Full connection (fully connected layer, hidden layer), create the classic ANN, Dense adds a fully connected layer
classifier.add(Dense(units= 128, activation='relu'))
classifier.add(Dropout(rate=random.uniform(0, .2)))

#Output layer, if more than two outcomes use softmax
classifier.add(Dense(units= 1, activation='sigmoid'))
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
# Image augmentation to help prevent overfitten because 10000 images isn't very many
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
        'dataset/training_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [9]:
classifier.fit_generator(
        training_set,
        samples_per_epoch=8000,
        epochs=10,
        validation_data=test_set,
        nb_val_samples=2000,
        workers=4,
        use_multiprocessing=True)

Epoch 1/10
250/250 [==============================] - 789s 3s/step - loss: 0.6941 - accuracy: 0.5156 - val_loss: 0.6794 - val_accuracy: 0.5600
Epoch 2/10
250/250 [==============================] - 779s 3s/step - loss: 0.6862 - accuracy: 0.5546 - val_loss: 0.6423 - val_accuracy: 0.5751
Epoch 3/10
250/250 [==============================] - 785s 3s/step - loss: 0.6813 - accuracy: 0.5589 - val_loss: 0.6503 - val_accuracy: 0.5782
Epoch 4/10
250/250 [==============================] - 707s 3s/step - loss: 0.6754 - accuracy: 0.5742 - val_loss: 0.6674 - val_accuracy: 0.6266
Epoch 5/10
250/250 [==============================] - 691s 3s/step - loss: 0.6630 - accuracy: 0.6058 - val_loss: 0.7015 - val_accuracy: 0.6205
Epoch 6/10
250/250 [==============================] - 649s 3s/step - loss: 0.6524 - accuracy: 0.6130 - val_loss: 0.7284 - val_accuracy: 0.6204
Epoch 7/10
250/250 [==============================] - 687s 3s/step - loss: 0.6408 - accuracy: 0.6348 - val_loss: 0.6239 - val_accuracy: 0.6246

In [77]:
classifier.save_weights('dog_cat_cnn.h5') # Save the model for later use
classifier.save('dog_cat_model.h5')


In [94]:

# how process individual images
import numpy as np
from keras.preprocessing import image
# classifier.load_weights('dog_cat_cnn.h5')
test_image = image.load_img('dataset/single_prediction/cat_or_dog_4.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = classifier.predict(test_image)
training_set.class_indices

if result[0][0] == 1:
    print('dog')
else:
    print('cat')

dog
